In [54]:
# Anpassung der Bibliothek Pybtex für den Harvard-Style

import re
import io
from pybtex.database import parse_string
from pybtex.plugin import find_plugin
from pybtex.style.formatting import BaseStyle, toplevel
from pybtex.style.template import (
    field, first_of, href, join, optional, optional_field, sentence, tag,
    together, words, node, FieldIsMissing
)
from pybtex import richtext
from pybtex.richtext import Text, Symbol

firstlast = find_plugin('pybtex.style.names', 'firstlast')()
lastfirst = find_plugin('pybtex.style.names', 'lastfirst')()

def format_pages(text):
    dash_re = re.compile(r'-+')
    pages = Text(Symbol('ndash')).join(text.split(dash_re))
    if re.search('[-‒–—―]', str(text)):
        return Text("pp.", Symbol('nbsp'), pages)
    return Text("p.", Symbol('nbsp'), pages)

pages = field('pages', apply_func=format_pages)
date = words[field('year'), optional[", ", field('month')]]

@node
def harv_names(children, context, role, **kwargs):
    """
    Returns formatted names as an APA compliant reference list citation.
    """
    assert not children

    try:
        persons = context['entry'].persons[role]
    except KeyError:
        raise FieldIsMissing(role, context['entry'])

    style = context['style']

    if len(persons) > 20:
        formatted_names = [style.format_name(
            person, style.abbreviate_names) for person in persons[:20]]
        formatted_names += [richtext.Text("et al.")]
        return join(sep=', ')[formatted_names].format_data(context)
    else:
        formatted_names = [style.format_name(
            person, style.abbreviate_names) for person in persons]
        return join(sep=', ', sep2=' & ', last_sep=', & ')[
            formatted_names].format_data(context)

@node
def editor_names(children, context, with_suffix=True, **kwargs):
    """
    Returns formatted editor names for inbook.
    """
    assert not children

    try:
        editors = context['entry'].persons['editor']
    except KeyError:
        raise FieldIsMissing('editor', context['entry'])

    formatted_names = [
        firstlast.format(editor, True) for editor in editors]

    if with_suffix:
        return words[
            join(sep=', ', sep2=', & ', last_sep=', & ')[formatted_names],
            "(eds.)" if len(editors) > 1 else "(ed.)"
        ].format_data(context)

    return join(sep=', ', sep2=', & ', last_sep=', & ')[
        formatted_names
    ].format_data(context)

class HarvStyle(BaseStyle):
    name = 'harv'
    default_name_style = 'lastfirst'
    default_sorting_style = 'author_year_title'
    #default_label_style = 'apa7'

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.abbreviate_names = True

    def format_names(self, role, as_sentence=False):
        formatted_names = harv_names(role)
        if as_sentence:
            return sentence(capfirst=False)[formatted_names]
        else:
            return formatted_names

    def format_editor(self, e, as_sentence=True):
        editors = self.format_names('editor', as_sentence=False)
        if 'editor' not in e.persons:
            # when parsing the template, a FieldIsMissing exception
            # will be thrown anyway; no need to do anything now,
            # just return the template that will throw the exception
            return editors
        if len(e.persons['editor']) > 1:
            word = 'eds.'
        else:
            word = 'ed.'
        result = join(sep=', ')[editors, word]
        if as_sentence:
            return sentence[result]
        else:
            return result

    def format_volume(self, e, for_article=False):
        prefix = "Vol."
        if for_article:
            return join[
                tag('em')[field('volume')],
                optional['(', field('number'), ')'],
            ]
        else:
            return optional[together[prefix, field('volume')]]

    def format_number(self, e):
        prefix = "No."
        return optional[together[prefix, field('number')]]

    def format_pages(self, e):
        prefix = "pp."
        return optional[together[prefix, field('pages')]]

    def format_title(self, e, which_field, as_sentence=False):
        formatted_title = field(
            which_field, apply_func=lambda text: text.capitalize()
        )
        if as_sentence:
            return sentence[formatted_title]
        else:
            return formatted_title

    def format_btitle(self, e, which_field, as_sentence=False):
        formatted_title = tag('em')[field(which_field)]
        if as_sentence:
            return sentence[formatted_title]
        else:
            return formatted_title

    def format_web_refs(self, e):
        return sentence(add_period=False)[
            optional[self.format_url(e)],
            optional[self.format_eprint(e)],
            optional[self.format_pubmed(e)],
            optional[self.format_doi(e)],
        ]

    def format_url(self, e):
        return words[
            'URL:',
            href[
                field('url', raw=True),
                field('url', raw=True)
            ]
        ]

    def format_pubmed(self, e):
        return href[
            join[
                'https://www.ncbi.nlm.nih.gov/pubmed/',
                field('pubmed', raw=True)
            ],
            join[
                'PMID:',
                field('pubmed', raw=True)
            ]
        ]

    def format_doi(self, e):
        return href[
            join[
                'https://doi.org/',
                field('doi', raw=True)
            ],
            join[
                'doi:',
                field('doi', raw=True)
            ]
        ]

    def format_eprint(self, e):
        return href[
            join[
                'https://arxiv.org/abs/',
                field('eprint', raw=True)
            ],
            join[
                'arXiv:',
                field('eprint', raw=True)
            ]
        ]

    def format_date(self, e):
        return sentence[
            first_of[optional[date], "n.d."]
        ]

    def get_article_template(self, e):
        volume_and_pages = first_of[
            optional[
                join(sep='')[
                    self.format_volume(e, for_article=True),
                    optional[': ',field('pages')]
                ]
            ],
            field('pages'),
        ]
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[field('year')],
            join['"',field('title'),'."'],
            sentence(sep=', ')[
                field('journal'),
                volume_and_pages
            ],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]

    def get_book_template(self, e):
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[field('year')],
            sentence[field('title')],
            sentence(sep=', ')[
                optional[self.format_editor(e)], 
                optional[field('series')],
                optional[self.format_volume(e)],
                optional[field('edition'), ' ed.'],
            ],
            sentence[
                join[optional[field('address'), ': '],
                    field('publisher')],
            ],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]

    def get_booklet_template(self, e):
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[join(sep=' ')[field('year'), optional['(',field('month'),')']]],
            sentence[join(sep=', ')[field('title'), 'brochure']],
            sentence[optional_field('howpublished')],
            sentence(sep=', ')[
                optional[self.format_editor(e)], 
                optional[field('series')],
                optional[self.format_volume(e)],
                optional[self.format_number(e)],
                optional[field('edition'), ' ed.'],
            ],
            sentence[
                join[optional[field('address'), ': '],
                    optional[field('organization')]],
            ],
            sentence[optional[self.format_pages(e)]],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]

    def get_inbook_template(self, e):
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[field('year')],
            sentence[
                words(sep=' ')[
                    join['"',field('title'),'"'],
                    'in',
                    optional[self.format_editor(e)],
                    self.format_btitle(e, 'booktitle')],
            ],
            sentence(sep=', ')[ 
                optional[field('series')],
                optional[self.format_volume(e)],
                optional[self.format_number(e)],
                optional[field('edition'), ' ed.'],
            ],
            sentence[
                join[optional[field('address'), ': '],
                    field('publisher')],
            ],        
            sentence[optional[self.format_pages(e)]],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]
    
    def get_incollection_template(self, e):
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[field('year')],
            sentence[
                words(sep=' ')[
                    join['"',field('title'),'"'],
                    'in',
                    optional[self.format_editor(e)],
                    self.format_btitle(e, 'booktitle')],
            ],
            sentence(sep=', ')[
                optional[field('type')],
                optional[field('series')],
                optional[self.format_volume(e)],
                optional[self.format_number(e)],
                optional[field('edition'), ' ed.'],
            ],
            sentence[
                join[optional[field('address'), ': '],
                    field('publisher')],
            ],        
            sentence[optional[self.format_pages(e)]],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]

    def get_inproceedings_template(self, e):
        if 'address' in e.fields and 'publisher' in e.fields:
            address_and_publisher = join[optional[field('address'), ': '],
                                        optional[field('publisher')]]
        elif 'address' in e.fields:
            address_and_publisher = optional[field('address')]
        else: address_and_publisher = None
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[join(sep=' ')[field('year'), optional['(',field('month'),')']]],
            sentence[
                words(sep=' ')[
                    join['"',field('title'),'"'],
                    'in',
                    optional[self.format_editor(e)],
                    self.format_btitle(e, 'booktitle')],
            ],
            sentence(sep=', ')[
                optional[field('series')],
                optional[self.format_volume(e)],
                optional[self.format_number(e)],
                optional[field('edition'), ' ed.'],
            ],
            sentence[address_and_publisher],        
            sentence[optional[self.format_pages(e)]],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]
            
    def get_manual_template(self, e):
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[join(sep=' ')[field('year'), optional['(',field('month'),')']]],
            sentence[field('title')],
            sentence(sep=', ')[
                optional[field('organization')],
                optional[field('address')],
            ],
            sentence[optional[self.format_pages(e)]],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]

    def get_misc_template(self, e):
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[join(sep=' ')[field('year'), optional['(',field('month'),')']]],
            sentence[field('title')],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]

    def get_phdthesis_template(self, e):
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[join(sep=' ')[field('year'), optional['(',field('month'),')']]],
            sentence[join['"',field('title'),'"']],
            sentence(sep=', ')[
                'Ph.D. thesis',
                join[optional[field('address'), ': '],
                    field('school')],
            ],
            sentence[optional[self.format_pages(e)]],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]

    def get_proceedings_template(self, e):
        if 'address' in e.fields and ('publisher' in e.fields or 'organization' in e.fields):
            address_and_publisher = join[optional[field('address'), ': '],
                                        join(sep=', ')[optional[field('publisher')], optional[field('organization')]]]
        elif 'address' in e.fields:
            address_and_publisher = optional[field('address')]
        else: address_and_publisher = None
        if 'editor' in e.persons:
            return toplevel[
                self.format_editor(e),
                sentence[join(sep=' ')[field('year'), optional['(',field('month'),')']]],
                sentence[field('title')],
                sentence(sep=', ')[
                    optional[field('series')],
                    optional[self.format_volume(e)],
                    optional[self.format_number(e)],
                    optional[field('edition'), ' ed.'],
                ],
                sentence[address_and_publisher],
                sentence[optional_field('note')],
                sentence[self.format_web_refs(e)]
            ]
        else:
            return toplevel[
                sentence[field('title')],
                sentence[join(sep=' ')[field('year'), optional['(',field('month'),')']]],
                sentence(sep=', ')[
                    optional[field('series')],
                    optional[self.format_volume(e)],
                    optional[self.format_number(e)],
                    optional[field('edition'), ' ed.'],
                ],
                sentence[address_and_publisher],
                sentence[optional_field('note')],
                sentence[self.format_web_refs(e)]
            ]

    def get_techreport_template(self, e):
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[join(sep=' ')[field('year'), optional['(',field('month'),')']]],
            sentence[join['"',field('title'),'"']],
            sentence[
                join[optional[field('address'), ': '],
                    field('institution')],
            ], 
            sentence[optional[self.format_pages(e)]],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]
       
    def get_unpublished_template(self, e):
        return toplevel[
            self.format_names('author', as_sentence=True),
            sentence[join(sep=' ')[field('year'), optional['(',field('month'),')']]],
            sentence[join['"',field('title'),'"']],
            sentence[
                join[optional[field('address'), ': '],
                    field('institution')],
            ], 
            sentence[optional[self.format_pages(e)]],
            sentence[optional_field('note')],
            sentence[self.format_web_refs(e)]
        ]

In [55]:
# Beispiel-BibTeX-Code
bibtex_code = """
@article{smith2023impact,
  author = {Jane Smith and John Doe},
  title = {The Impact of Sample Studies on Research Methods},
  journal = {Journal of Example Studies},
  year = {2023},
  month = {May},
  volume = {15},
  number = {3},
  pages = {123-145},
  note = {Special Issue on Methodologies}
}


"""

# BibTeX-Code parsen
bib_data = parse_string(bibtex_code, 'bibtex')
#parser = bibtex.Parser()
#bib_data = parser.parse_file("mybib.bib")

# Erstellen Sie eine Instanz der MLA-Stilklasse
harv_style = HarvStyle()
plain_backend = find_plugin('pybtex.backends', 'text')()

# Einträge formatieren
formatted_entries = harv_style.format_entries(bib_data.entries.values())

# Custom Backend zum Erfassen der Ausgabe in einem String
class StringBackend(plain_backend.__class__):
    def __init__(self):
        super().__init__()
        self.output = io.StringIO()

    def write_entry(self, label, text):
        self.output.write(text + "\n")

    def get_output(self):
        return self.output.getvalue()

# Custom Backend verwenden
string_backend = StringBackend()
try:
    for entry in formatted_entries:
        string_backend.write_entry(entry.key, entry.text.render(string_backend))
except FieldIsMissing as ex:
    print(ex)

# Formatierten String erhalten
formatted_string = string_backend.get_output()
print(formatted_string)

Smith, J. & Doe, J. 2023. "The Impact of Sample Studies on Research Methods." Journal of Example Studies, 15(3): 123-145. Special Issue on Methodologies.

